## Libraries

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
#nltk.download('stopwords')
import re
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pyLDAvis
import pandas as pd
import gensim as gs
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import tmplot

/home/simone/anaconda3/envs/nlp/lib/python3.9/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


## Data loading

In [2]:
# the data in Pandas DF
df = pd.read_json('../sampleData/beerReviews/product_reviews.json')
# a list of reviews
docs = df.loc[df["text"].notnull(), "text"].to_list()

## Text pre-processing


In [3]:
# stemming
porter_stemmer = nltk.PorterStemmer().stem
docs = [porter_stemmer(doc) for doc in docs]

In [4]:
# tokenization
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()
    docs[idx] = tokenizer.tokenize(docs[idx])
# remove numbers, but not words that contain numbers
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]
# remove words that are only one character
docs = [[token for token in doc if len(token) > 1] for doc in docs]

NameError: name 'tokenizer' is not defined

In [ ]:
docs

[['actually',
  'liked',
  'this',
  'beer',
  'for',
  'precisely',
  'the',
  'same',
  'reason',
  'many',
  'seem',
  'to',
  'have',
  'not',
  'liked',
  'it',
  'while',
  'the',
  'can',
  'does',
  'say',
  'new',
  'england',
  'style',
  'ipa',
  'you',
  'still',
  'must',
  'rate',
  'the',
  'beer',
  'individually',
  'not',
  'on',
  'some',
  'arbitrary',
  'idea',
  'of',
  'what',
  'you',
  'think',
  'it',
  'should',
  'be',
  'because',
  'of',
  'the',
  'style',
  'it',
  'certainly',
  'not',
  'what',
  've',
  'come',
  'to',
  'expect',
  'from',
  'the',
  'style',
  'but',
  'also',
  'much',
  'more',
  'of',
  'west',
  'coast',
  'ipa',
  'fan',
  'and',
  'this',
  'almost',
  'felt',
  'like',
  'it',
  'was',
  'right',
  'in',
  'the',
  'middle',
  'juicier',
  'and',
  'and',
  'hazier',
  'than',
  'typical',
  'wcipa',
  'but',
  'more',
  'bitter',
  'and',
  'dry',
  'than',
  'typical',
  'neipa',
  'personally',
  'really',
  'enjoyed',
  '

In [5]:
# stop-word removal
english_stops = set(porter_stemmer(w) for w in stopwords.words('english'))
docs = [[token for token in doc if token not in english_stops] for doc in docs]

## Model setup

In [7]:
# create a dictionary representation of the documents
dictionary = Dictionary(docs)
# create a corpus of vectorized docs
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [8]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 105
Number of documents: 8524


In [9]:
# training parameters
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

In [10]:
# make an index to word dictionary
temp = dictionary[0]
id2word = dictionary.id2token

In [11]:

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

## Visualization

In [ ]:
phi = tmplot.get_phi(model)

In [ ]:
theta = tmplot.get_theta(model, corpus=corpus)

In [ ]:
tmplot.get_top_docs(docs, model=model, corpus=corpus)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,"[served, bright, slightly, haze, copper, orang...","[pack, total, wine, always, try, review, beer,...","[courtesy, jasonja1474]","[terrible, brew, sweet, sickening, monkey, puk...","[ounce, tulip, glass, best, before, pours, lig...","[cerveza, obscura, con, maltas, caramelizadas,...","[dorada, con, un, dedo, de, espuma, de, buena,...","[pours, light, fizzy, head, fades, semi, quick...","[de, labirratorium, en, copa, teku, jugando, f...","[milky, tan, head, mint, lime, sorbet, wheaty,..."
1,"[appearance, copper, color, touch, haze, lots,...","[drank, bottle, appearance, noted, beer, poor,...","[bottle, room, temp, wax, was, big, bugger, in...",[refresh],"[ml, bottle, weizen, glass, bottle, dating, po...","[does, this, dunkel, make, raul, want, dunk, n...","[de, color, dorado, con, un, dedo, de, espuma,...","[many, thanks, jhartley, this, dark, brow, bla...","[de, labirratorium, en, copa, hefe, de, bavari...","[thanks, piclee, sending, this, heretofore, un..."
2,"[pours, deep, copper, color, creamy, white, he...","[noticed, cans, this, product, cbc, massachuse...","[750ml, bottle, got, bright, orange, clear, ni...","[foggy, amber, brown, fairly, fluid, surface, ...","[ounce, tulip, glass, dating, recent, release,...","[got, this, beer, rvdoorn, returned, his, holi...","[bouteille, de, 341ml, quel, beau, produit, be...","[pours, foggy, toffee, brown, finger, khaki, f...","[foul, spoiled, beer, hopelessly, corrupted, b...","[draught, brien, san, diego, beer, resides, gl..."
3,"[good, solid, ipa, pours, pint, glass, nice, c...","[major, sucker, fruit, beer, try, this, one, w...","[poured, gently, 750ml, bottle, long, stemmed,...","[12oz, bottle, tulip, why, keep, buying, fruit...","[ounce, tulip, glass, canned, pours, lightly, ...","[cypress, honey, lager, looks, lot, like, hone...","[de, color, rojizo, con, dos, dedos, de, espum...","[tap, pours, murky, brighter, brown, salted, c...","[poured, tap, saraveza, portland, during, elys...","[dave, hom]"
4,"[poured, oz, bottle, pint, glass, appearance, ...","[one, kind, first, review, prefer, ipa, count,...","[barrel, vanilla, shine, huge, vanilla, fan, f...","[smooth, light, taste, refresh]","[ounce, tulip, glass, canned, pours, slightly,...","[first, this, unibroue, intents, purposes, got...","[de, alcampo, en, copa, teku, tras, cena, chus...","[supposed, share, rudzud, enjoyed, during, bru...","[first, tap, city, tap, house, philly, pa, rev...","[review, beeradvocate, magazine, issue, feb, d..."
